In [2]:
from typing import *
import wandb
import numpy as np

import llminference as L

api = wandb.Api()
def get_run(id: str) -> wandb.apis.public.Run:
    return api.run(f"research/sparse-attention/runs/{id}")

In [76]:
# From Figures.ipynb
run_ids = {'cnn_dailymail': ['jhtdsu6v', '3hdqdvcz', 'urvfe4ei', 'mut4eolt'],
 'repetition': ['9j21fmm7', 'tjca3tkr', '16ku5vpv', 'zubzr9ia'],
 'squad': ['e7v9cgwr', '0coy8ku5', '8fizgrac', '87whg2yy'],
 'triviaqa': ['zv7j4ilf', 'wwjvwthe', '8rr6ck8u', 'vswqd0gn'],
 'wikitext_bpc': ['l3ljj1h2', 'ulnw8eij', '37tikal9', 'mqeq0k3b']}

display_name = dict(
    dense="DENSE",
    ann="SPARQ",
    eviction="H2O",
    local="LM-INFINITE",
)

for _, ids in run_ids.items():
    ids.sort(key=lambda id: list(display_name.keys()).index(get_run(id).config["sparsity"]["name"]))

In [178]:
def shorten(text: str, length: int) -> str:
    return text if len(text) <= length else text[:length] + "..."

def show_example(example: Dict[str, Any], task: str):
    results = []
    for id in run_ids[task]:
        m = [r for r in get_run(id).summary.results if r["id"] == example["id"]]
        if len(m) > 2 if task == "triviaqa" else len(m) != 1:
            raise ValueError(f"Multiple matching results for {example['id']}")
        results.append(m[0])

    key = dict(squad="match", triviaqa="match", cnn_dailymail="rougeL", wikitext_bpc="bpc", repetition="match_length_char")[task]
    print({get_run(id).config["sparsity"]["name"]: r[key] for id, r in zip(run_ids[task], results)})
    print()
    print(f"### PROMPT ({example['id']})")
    if task == "cnn_dailymail":
        print(shorten(example["context"], 80) + L.tasks.summarisation.DEFAULT_PROMPT)
    elif task == "repetition":
        offset = example["context"].index(example["prompt"])
        lines = example["context"].split("\n")
        line_offset = np.cumsum([len(line) + 1 for line in lines])
        line_index = (line_offset < offset).sum()
        print("\n".join(lines[:2] + ["...", "..."] + lines[line_index:][:10] + ["...", "..."])
              + L.tasks.repetition.PROMPT_PREFIX + example["prompt"])
    elif task == "triviaqa":
        lines = example["context"].replace("\n\n", "\n").split("\n")
        print("\n".join(shorten(s, 50) for s in lines[:5] + ["...", "..."] + lines[-5:]))
        print(example["prompt"])
    elif task == "squad":
        lines = example["context"].split("\n")
        print("\n".join(shorten(s, 50) for s in lines[:-1]) + "\n" + lines[-1])
        print(example["prompt"])
    print()
    print("### OUTPUT")
    for id, result in zip(run_ids[task], results):
        out = shorten(result["output"].split("\n")[0], 75)
        print(f'{display_name[get_run(id).config["sparsity"]["name"]]:>11}: {out}')


# SQuAD

In [78]:
index = 51  # 26, 51
show_example(
    L.tasks.qa.add_few_shot_prompt(L.tasks.qa.SQuAD.data()[index], k=1, prompt_template=L.tasks.qa.get_default_prompt_template("meta-llama/Llama-2-13b-hf", shots=1)),
    "squad",
)

{'dense': True, 'ann': True, 'eviction': False, 'local': False}

### PROMPT (5725e1c4271a42140099d2d9)
Title: University of Chicago. Background: Current ...
Title: Harvard University. Background: Harvard has...
Title: Oxygen. Background: In one experiment, Lavo...
Title: Oxygen. Background: Oxygen storage methods ...
Title: Fresno, California. Background: This vibran...
Title: Fresno, California. Background: Before Worl...
Title: Steam engine. Background: The working fluid...
Title: Sky (United Kingdom). Background: While BSk...
From what you've just read about Fresno, California, please answer the following questions.
Question: Where is Audra McDonald from?
Answer: Fresno
Question: In what year did Roger Rocka's Dinner Theater & Good Company Players open?
Answer:

### OUTPUT
      DENSE: 1978
      SPARQ: 1978
        H2O: 1979
LM-INFINITE: 1975 (Roger Rock


# TriviaQA

In [152]:
index = 9  # 22
example = L.tasks.qa.add_few_shot_prompt(L.tasks.qa.TriviaQA.data(context="wiki")[index], k=0, prompt_template=L.tasks.qa.get_default_prompt_template("meta-llama/Llama-2-13b-hf", shots=0))
show_example(example, "triviaqa")

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/19 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/47 [00:00<?, ?it/s]

{'dense': True, 'ann': True, 'eviction': True, 'local': False}

### PROMPT (dpql_5685)
Apéritifs and digestifs ( and) are drinks, typical...
Apéritifs
An apéritif is an alcoholic beverage usually serve...
"Apéritif" may also refer to a snack that precedes...
"Apéritif" is a French word derived from the Latin...
...
...
* Distilled liquors (ouzo, tequila, whisky or akva...
* Liquor cocktails (Black Russian, Rusty Nail, etc...
In certain areas, it is not uncommon for a digesti...
Bitter digestifs typically contain carminative her...
In many countries, people drink alcoholic beverage...
Question: Which aperitif is named for the Paris chemist who created it in 1846?
Answer:

### OUTPUT
      DENSE: Dubonnet
      SPARQ: Dubonnet
        H2O: Dubonnet
LM-INFINITE: Byrrh


# CNN/DailyMail

In [176]:
index = 18  # 7 9
show_example(L.tasks.summarisation.CnnDailymail.data()[index], "cnn_dailymail")

{'dense': 0.2653061224489796, 'ann': 0.2653061224489796, 'eviction': 0.24242424242424243, 'local': 0.24489795918367346}

### PROMPT (a62bbf503be06e8b1f8baa4f3cd537310d5aa3bc)
Article: Prince William arrived in China tonight for one of the most high-profil...
Summary:

### OUTPUT
      DENSE: Prince William arrived in China tonight for one of the most high-profile – ...
      SPARQ: Prince William arrived in China tonight for one of the most high-profile – ...
        H2O: Prince William arrived in China tonight for a high-profile visit that will ...
LM-INFINITE: Prince William and Kate Middleton are in Japan for a three-day tour. The ro...


# Repetition

In [194]:
index = 14  # 0, 7, 9
show_example(L.tasks.repetition.Shakespeare.data()[index], "repetition")

{'dense': 257, 'ann': 162, 'eviction': 4, 'local': 0}

### PROMPT (210496)
 you mistake me much;
I do lament the sickness of the king.
...
...
Peace, children, peace! the king doth love you well:
Incapable and shallow innocents,
You cannot guess who caused your father's death.

Boy:
Grandam, we can; for my good uncle Gloucester
Told me, the king, provoked by the queen,
Devised impeachments to imprison him :
And when my uncle told me so, he wept,
And hugg'd me in his arm, and kindly kiss'd my cheek;
...
...
 the king doth love you well:
Incapable and shallow innocents,
You cannot guess who caused your father's death.

Boy:
Grandam, we

### OUTPUT
      DENSE: can; for my good uncle Gloucester
      SPARQ: can; for my good uncle Gloucester
        H2O: can;
LM-INFINITE: 'll not stand to prate, but to the purpose.


# WikiText BPC

In [198]:
index = 2  # 1
example = L.tasks.bpc.WikiText.data()[index]
lines = example["prefill"].split("\n")
print(f"\\begin{{verbatim}}### QUERY ({index})")
print()
print("\n".join([shorten(line, 80) for line in lines[:-1]]))
print("\\end{verbatim}")
print("\\" + lines[-1] + "{\color{blue}\\" + example["reference"] + "}")
print("\\begin{verbatim}### BPC")
for id in run_ids["wikitext_bpc"]:
    run = get_run(id)
    bpc = run.summary.results[index]["bpc"]
    print(f'{display_name[get_run(id).config["sparsity"]["name"]]:>11}: {bpc:.3f}')
print("\\end{verbatim}")

Repo card metadata block was not found. Setting CardData to empty.


\begin{verbatim}### QUERY (2)

 = Mellor hill fort = 
 
 Mellor hill fort is a prehistoric site in North West England , that dates from ...
 
 = = Location = = 
 
 Mellor lies on the western edge of the Peak District in the Metropolitan Boroug...
 
 = = Background = = 
 
 Until the 19th century little was known about hill forts ; none had been excava...
 The study of hill forts was popular in the 19th century , with a revival in the...
 
 = = History = = 
 
 There is evidence of human activity on the site pre @-@ dating the Iron Age , a...
 A flint dagger was discovered on the site . This type of artefact is rare in Gr...
 The hill fort was built in and used throughout the Iron Age , as demonstrated b...
\end{verbatim}
\ Fragments of glass , possibly Roman in origin , and shards of pottery which date to the 1st and 2nd centuries AD , indicate the site was used in the Romano @-@ British period . However{\color{blue}\ no Roman structures have been discovered , and the nature of Roman act